# JSON Module

---

## Table of Contents
1. Introduction to JSON
2. Encoding (Python to JSON)
3. Decoding (JSON to Python)
4. Working with Files
5. Custom Encoding/Decoding
6. Pretty Printing
7. Common Patterns
8. Key Points
9. Practice Exercises

---

## 1. Introduction to JSON

JSON (JavaScript Object Notation) is a lightweight data interchange format.

In [ ]:
import json

# JSON data types and Python equivalents
print("JSON to Python type mapping:")
print("  object  -> dict")
print("  array   -> list")
print("  string  -> str")
print("  number  -> int/float")
print("  true    -> True")
print("  false   -> False")
print("  null    -> None")

---

## 2. Encoding (Python to JSON)

In [ ]:
# json.dumps() - Convert Python object to JSON string
data = {
    "name": "Alice",
    "age": 30,
    "is_student": False,
    "courses": ["Python", "Data Science"],
    "address": None
}

json_string = json.dumps(data)
print(f"JSON string: {json_string}")
print(f"Type: {type(json_string)}")

In [ ]:
# Encoding different types
print(json.dumps("Hello"))        # String
print(json.dumps(42))             # Integer
print(json.dumps(3.14))           # Float
print(json.dumps(True))           # Boolean
print(json.dumps(None))           # None
print(json.dumps([1, 2, 3]))      # List
print(json.dumps({"a": 1}))       # Dict

In [ ]:
# dumps() options
data = {"name": "Bob", "scores": [85, 90, 78]}

# indent for pretty printing
print("With indent:")
print(json.dumps(data, indent=2))

# sort_keys
print("\nWith sorted keys:")
print(json.dumps(data, sort_keys=True))

In [ ]:
# separators option
data = {"a": 1, "b": 2}

# Default separators
print(f"Default: {json.dumps(data)}")

# Compact (no spaces)
print(f"Compact: {json.dumps(data, separators=(',', ':'))}")

# Custom separators
print(f"Custom: {json.dumps(data, separators=(', ', ' = '))}")

In [ ]:
# ensure_ascii option
data = {"name": "Cafe", "symbol": "Euro"}

print(f"ASCII (default): {json.dumps(data)}")
print(f"Unicode: {json.dumps(data, ensure_ascii=False)}")

---

## 3. Decoding (JSON to Python)

In [ ]:
# json.loads() - Parse JSON string to Python object
json_string = '{"name": "Alice", "age": 30, "active": true}'

data = json.loads(json_string)
print(f"Python object: {data}")
print(f"Type: {type(data)}")
print(f"Name: {data['name']}")

In [ ]:
# Decoding different types
print(json.loads('"Hello"'))      # String
print(json.loads('42'))           # Integer
print(json.loads('3.14'))         # Float
print(json.loads('true'))         # Boolean -> True
print(json.loads('null'))         # null -> None
print(json.loads('[1, 2, 3]'))    # Array -> List

In [ ]:
# Handling parse errors
invalid_json = '{name: "Alice"}'

try:
    data = json.loads(invalid_json)
except json.JSONDecodeError as e:
    print(f"JSON Error: {e}")
    print(f"Position: {e.pos}")
    print(f"Line: {e.lineno}, Column: {e.colno}")

In [ ]:
# parse_float and parse_int options
from decimal import Decimal

json_string = '{"price": 19.99, "quantity": 5}'

# Default parsing
data1 = json.loads(json_string)
print(f"Default: {data1}, price type: {type(data1['price'])}")

# Parse floats as Decimal
data2 = json.loads(json_string, parse_float=Decimal)
print(f"Decimal: {data2}, price type: {type(data2['price'])}")

---

## 4. Working with Files

In [ ]:
# json.dump() - Write to file
data = {
    "users": [
        {"name": "Alice", "age": 30},
        {"name": "Bob", "age": 25}
    ]
}

# Write to file
with open('data.json', 'w') as f:
    json.dump(data, f, indent=2)

print("Data written to data.json")

In [ ]:
# json.load() - Read from file
with open('data.json', 'r') as f:
    loaded_data = json.load(f)

print(f"Loaded data: {loaded_data}")
print(f"First user: {loaded_data['users'][0]}")

In [ ]:
# Clean up
import os
if os.path.exists('data.json'):
    os.remove('data.json')
    print("Cleaned up data.json")

---

## 5. Custom Encoding/Decoding

In [ ]:
# Problem: Some types are not JSON serializable
from datetime import datetime

data = {"timestamp": datetime.now()}

try:
    json.dumps(data)
except TypeError as e:
    print(f"Error: {e}")

In [ ]:
# Solution 1: default function
def json_serializer(obj):
    if isinstance(obj, datetime):
        return obj.isoformat()
    raise TypeError(f"Object of type {type(obj)} is not JSON serializable")

data = {"timestamp": datetime.now()}
json_string = json.dumps(data, default=json_serializer)
print(f"Serialized: {json_string}")

In [ ]:
# Solution 2: Custom JSONEncoder class
class CustomEncoder(json.JSONEncoder):
    def default(self, obj):
        if isinstance(obj, datetime):
            return {"_type": "datetime", "value": obj.isoformat()}
        if isinstance(obj, set):
            return {"_type": "set", "value": list(obj)}
        return super().default(obj)

data = {
    "timestamp": datetime.now(),
    "tags": {"python", "json", "tutorial"}
}

json_string = json.dumps(data, cls=CustomEncoder, indent=2)
print(json_string)

In [ ]:
# Custom decoder with object_hook
def custom_decoder(obj):
    if "_type" in obj:
        if obj["_type"] == "datetime":
            return datetime.fromisoformat(obj["value"])
        if obj["_type"] == "set":
            return set(obj["value"])
    return obj

# Decode back
decoded = json.loads(json_string, object_hook=custom_decoder)
print(f"Decoded: {decoded}")
print(f"timestamp type: {type(decoded['timestamp'])}")
print(f"tags type: {type(decoded['tags'])}")

In [ ]:
# Encoding custom classes
class Person:
    def __init__(self, name, age):
        self.name = name
        self.age = age
    
    def to_dict(self):
        return {"_type": "Person", "name": self.name, "age": self.age}
    
    @classmethod
    def from_dict(cls, data):
        return cls(data["name"], data["age"])

# Encoder
def encode_person(obj):
    if isinstance(obj, Person):
        return obj.to_dict()
    raise TypeError(f"Cannot serialize {type(obj)}")

# Decoder
def decode_person(obj):
    if obj.get("_type") == "Person":
        return Person.from_dict(obj)
    return obj

# Test
person = Person("Alice", 30)
json_str = json.dumps(person, default=encode_person)
print(f"Encoded: {json_str}")

decoded_person = json.loads(json_str, object_hook=decode_person)
print(f"Decoded: {decoded_person.name}, {decoded_person.age}")

---

## 6. Pretty Printing

In [ ]:
# Pretty print with indent
data = {
    "company": "Tech Corp",
    "employees": [
        {"name": "Alice", "role": "Developer"},
        {"name": "Bob", "role": "Designer"}
    ],
    "active": True
}

print("indent=2:")
print(json.dumps(data, indent=2))

print("\nindent=4:")
print(json.dumps(data, indent=4))

In [ ]:
# Using pprint for complex objects
from pprint import pprint

# First parse JSON, then pprint
json_str = '{"a": {"b": {"c": [1, 2, 3]}}}'
data = json.loads(json_str)

print("pprint output:")
pprint(data)

---

## 7. Common Patterns

In [ ]:
# Pattern 1: Safe JSON loading
def safe_load_json(json_string, default=None):
    try:
        return json.loads(json_string)
    except json.JSONDecodeError:
        return default

print(safe_load_json('{"valid": true}'))
print(safe_load_json('invalid json', default={}))

In [ ]:
# Pattern 2: Config file handler
class ConfigManager:
    def __init__(self, filepath):
        self.filepath = filepath
        self.config = {}
    
    def load(self):
        try:
            with open(self.filepath, 'r') as f:
                self.config = json.load(f)
        except FileNotFoundError:
            self.config = {}
        return self.config
    
    def save(self):
        with open(self.filepath, 'w') as f:
            json.dump(self.config, f, indent=2)
    
    def get(self, key, default=None):
        return self.config.get(key, default)
    
    def set(self, key, value):
        self.config[key] = value

# Usage example (not running to avoid file creation)
print("ConfigManager pattern demonstrated")

In [ ]:
# Pattern 3: JSON Lines (JSONL) format
# Each line is a separate JSON object

records = [
    {"id": 1, "name": "Alice"},
    {"id": 2, "name": "Bob"},
    {"id": 3, "name": "Charlie"}
]

# Write JSONL
jsonl_string = '\n'.join(json.dumps(r) for r in records)
print("JSONL format:")
print(jsonl_string)

# Read JSONL
print("\nParsed records:")
for line in jsonl_string.split('\n'):
    record = json.loads(line)
    print(f"  {record}")

In [ ]:
# Pattern 4: Deep merge JSON objects
def deep_merge(base, override):
    result = base.copy()
    for key, value in override.items():
        if key in result and isinstance(result[key], dict) and isinstance(value, dict):
            result[key] = deep_merge(result[key], value)
        else:
            result[key] = value
    return result

base_config = {"db": {"host": "localhost", "port": 5432}, "debug": False}
override = {"db": {"port": 3306}, "debug": True}

merged = deep_merge(base_config, override)
print(json.dumps(merged, indent=2))

---

## 8. Key Points

1. **json.dumps()**: Python object to JSON string
2. **json.loads()**: JSON string to Python object
3. **json.dump()**: Write to file
4. **json.load()**: Read from file
5. **indent**: Pretty print with indentation
6. **default**: Handle non-serializable types
7. **object_hook**: Custom decoding
8. **JSONEncoder**: Custom encoder class
9. **JSONDecodeError**: Handle parse errors

---

## 9. Practice Exercises

In [ ]:
# Exercise 1: Validate JSON string
# Return True if valid JSON, False otherwise

def is_valid_json(s):
    pass

# Test: is_valid_json('{"a": 1}'), is_valid_json('{invalid}')

In [ ]:
# Exercise 2: Flatten nested JSON
# {"a": {"b": 1}} -> {"a.b": 1}

def flatten_json(obj, prefix=''):
    pass

# Test: flatten_json({"a": {"b": {"c": 1}}})

In [ ]:
# Exercise 3: JSON diff
# Compare two JSON objects and return differences

def json_diff(obj1, obj2):
    pass

# Test with two similar but different dicts

In [ ]:
# Exercise 4: JSON schema validator (simple)
# Validate that required keys exist

def validate_schema(data, required_keys):
    pass

# Test: validate_schema({"name": "Alice"}, ["name", "age"])

In [ ]:
# Exercise 5: JSON to CSV converter
# Convert list of dicts to CSV string

def json_to_csv(data):
    pass

# Test: json_to_csv([{"name": "Alice", "age": 30}, {"name": "Bob", "age": 25}])

---

## Solutions

In [ ]:
# Solution 1:
def is_valid_json(s):
    try:
        json.loads(s)
        return True
    except json.JSONDecodeError:
        return False

print(f"Valid: {is_valid_json('{"a": 1}')}")
print(f"Invalid: {is_valid_json('{invalid}')}")

In [ ]:
# Solution 2:
def flatten_json(obj, prefix=''):
    result = {}
    for key, value in obj.items():
        new_key = f"{prefix}.{key}" if prefix else key
        if isinstance(value, dict):
            result.update(flatten_json(value, new_key))
        else:
            result[new_key] = value
    return result

nested = {"a": {"b": {"c": 1}}, "d": 2}
print(flatten_json(nested))

In [ ]:
# Solution 3:
def json_diff(obj1, obj2):
    diff = {"added": {}, "removed": {}, "changed": {}}
    
    all_keys = set(obj1.keys()) | set(obj2.keys())
    
    for key in all_keys:
        if key not in obj1:
            diff["added"][key] = obj2[key]
        elif key not in obj2:
            diff["removed"][key] = obj1[key]
        elif obj1[key] != obj2[key]:
            diff["changed"][key] = {"from": obj1[key], "to": obj2[key]}
    
    return diff

obj1 = {"a": 1, "b": 2, "c": 3}
obj2 = {"a": 1, "b": 20, "d": 4}
print(json.dumps(json_diff(obj1, obj2), indent=2))

In [ ]:
# Solution 4:
def validate_schema(data, required_keys):
    missing = [key for key in required_keys if key not in data]
    if missing:
        return False, f"Missing keys: {missing}"
    return True, "Valid"

data = {"name": "Alice"}
print(validate_schema(data, ["name"]))
print(validate_schema(data, ["name", "age"]))

In [ ]:
# Solution 5:
def json_to_csv(data):
    if not data:
        return ""
    
    headers = list(data[0].keys())
    lines = [','.join(headers)]
    
    for row in data:
        values = [str(row.get(h, '')) for h in headers]
        lines.append(','.join(values))
    
    return '\n'.join(lines)

data = [{"name": "Alice", "age": 30}, {"name": "Bob", "age": 25}]
print(json_to_csv(data))